In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments,EarlyStoppingCallback
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import wandb

### Check GPU

In [2]:
# Kiểm tra GPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


### Load Dataset

In [3]:
# Load dữ liệu
def load_data(file_path):
    label_map = {"Positive": 0, "Negative": 1, "Neutral": 2}
    df = pd.read_csv(file_path)
    texts = df['comment'].tolist()
    labels = df['label'].map(label_map).tolist()  # Encode nhãn
    return texts, labels

In [4]:
train_texts, train_labels = load_data("/kaggle/input/data-sentiment/train_processed.csv")
val_texts, val_labels = load_data("/kaggle/input/data-sentiment/val_processsed.csv")
test_texts, test_labels = load_data("/kaggle/input/data-sentiment/test_processsed.csv")

### Tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-cased")

def encode_data(texts, labels, tokenizer, max_length=512):
    inputs = tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )
    return inputs['input_ids'], inputs['attention_mask'], torch.tensor(labels)

train_inputs, train_masks, train_labels = encode_data(train_texts, train_labels, tokenizer)
val_inputs, val_masks, val_labels = encode_data(val_texts, val_labels, tokenizer)
test_inputs, test_masks, test_labels = encode_data(test_texts, test_labels, tokenizer)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

### Create Dataset Sentiment

In [6]:
# Tạo Dataset class
class SentimentDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = SentimentDataset(train_inputs, train_masks, train_labels)
val_dataset = SentimentDataset(val_inputs, val_masks, val_labels)
test_dataset = SentimentDataset(test_inputs, test_masks, test_labels)

### Load Model Bert

In [7]:
# Load BERT model
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-multilingual-cased", num_labels=3)

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Compute Metrics

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(axis=-1)  

    # Tính toán các chỉ số
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')  
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')

    # Trả về các chỉ số trong dạng dictionary
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1-score": f1
    }

### Hyperparameter Tuning

In [9]:
# Cài đặt TrainingArguments
wandb.login(key = '8a5cbfdaa29778a896996cc679358b1d96cf66b0')
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2 ,early_stopping_threshold=0.001)
training_args = TrainingArguments(
   output_dir="./bert_question_answer",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=34,
        per_device_eval_batch_size=34,
        gradient_accumulation_steps=8,
        num_train_epochs=15,
        disable_tqdm=False,
        weight_decay=0.25,
        save_total_limit=3,
        optim="adamw_hf",
        fp16=True,
        max_grad_norm=0.6,
        warmup_ratio=0.2,
        group_by_length=True,
        report_to="wandb",
        load_best_model_at_end=True,
        label_names=['labels'],
        lr_scheduler_type="linear"
)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Fine-Tuning

In [10]:
# Sử dụng Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)

# Huấn luyện model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: hdang1696 (hdang1696-no-work-experience). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.7
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241211_083151-nusqi7gg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./bert_question_answer
wandb: ⭐️ View project at https://wandb.ai/hdang1696-no-work-experience/huggingface
wandb: 🚀 View run at https://wandb.ai/hdang1696-n

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score
0,1.068500,0.943052,0.525180,0.275814,0.525180,0.361680
1,0.900600,0.762429,0.696043,0.662260,0.696043,0.661263
2,0.683900,0.594380,0.785971,0.710636,0.785971,0.744199
3,0.523200,0.505960,0.811151,0.830471,0.811151,0.767440
4,0.504600,0.478614,0.826439,0.818483,0.826439,0.797692
5,0.453800,0.479539,0.817446,0.792096,0.817446,0.791766
6,0.421200,0.474275,0.825540,0.810020,0.825540,0.798948
7,0.364300,0.487426,0.819245,0.807580,0.819245,0.812239
8,0.356400,0.482630,0.823741,0.810299,0.823741,0.815093


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer.tokenizer is now deprecated. You 

TrainOutput(global_step=128, training_loss=0.5840489119291306, metrics={'train_runtime': 3910.3962, 'train_samples_per_second': 29.867, 'train_steps_per_second': 0.054, 'total_flos': 1.8356896928047104e+16, 'train_loss': 0.5840489119291306, 'epoch': 8.97391304347826})

### Evaluation

In [11]:
# Đánh giá trên tập test
test_results = trainer.evaluate(test_dataset)
print(f"Test Results: {test_results}")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Test Results: {'eval_loss': 0.43345212936401367, 'eval_accuracy': 0.8462230215827338, 'eval_precision': 0.829242436606982, 'eval_recall': 0.8462230215827338, 'eval_f1-score': 0.817655362451093, 'eval_runtime': 43.7089, 'eval_samples_per_second': 50.882, 'eval_steps_per_second': 0.755, 'epoch': 8.97391304347826}


### Save Model

In [12]:
# Lưu mô hình
model.save_pretrained("bert_sentiment_model")
tokenizer.save_pretrained("bert_sentiment_model")

('bert_sentiment_model/tokenizer_config.json',
 'bert_sentiment_model/special_tokens_map.json',
 'bert_sentiment_model/vocab.txt',
 'bert_sentiment_model/added_tokens.json',
 'bert_sentiment_model/tokenizer.json')